In [1]:
library(RPostgreSQL)
library(tableone)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <-
"select hadm_id, icustay_id, echo,
    intime, outtime, echo_time, deathtime,
    norepinephrine_max, dobutamine_flag,
    vasofreeday28, ventfreeday28,
    sofa, sofa_drop_2, sofa_drop_3, day1, day2, day3,
    down2, down3, lactate_diff, creatinine_diff
from (select hadm_id, icustay_id, echo, intime, outtime, echo_time, deathtime from merged_data) co
left join subgroup using (hadm_id, icustay_id, echo)
left join lactate_diff using (hadm_id)
left join creatinine_diff using (hadm_id)
left join (select icustay_id, day1, day2, day3, down2, down3 from fluid) fl using (icustay_id)"

In [5]:
subgroup <- dbGetQuery(con, sql)
head(subgroup)

hadm_id,icustay_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,dobutamine_flag,vasofreeday28,⋯,sofa,sofa_drop_2,sofa_drop_3,day1,day2,day3,down2,down3,lactate_diff,creatinine_diff
159895,228416,1,2151-06-29 01:18:27,2151-07-27 16:00:56,2151-06-30,2151-09-18 00:00:00,0.000000,0,28.00000,⋯,3,1,0,-2110.000,-1930.000,-1475.000,-180.000,-635.000,-0.2,-0.7
169700,278148,0,2197-01-09 00:45:42,2197-01-10 06:46:29,NA,2197-01-10 03:15:00,0.000000,0,0.00000,⋯,8,-16,8,NA,NA,NA,NA,NA,NA,NA
103722,274174,1,2181-09-08 23:43:13,2181-09-26 16:52:02,2181-09-11,2182-11-10 00:00:00,0.000000,0,28.00000,⋯,5,3,2,-300.000,-1827.000,-10.000,1527.000,-290.000,NA,0.5
170515,214954,1,2176-03-11 04:58:23,2176-03-15 18:00:48,2176-03-11,NA,7.977352,0,26.04167,⋯,8,4,5,4299.590,2899.645,-1283.263,1399.945,5582.853,NA,0.2
183493,222457,0,2105-06-04 20:57:55,2105-06-06 20:01:51,NA,NA,0.000000,0,28.00000,⋯,1,0,1,5988.072,2102.761,NA,3885.311,NA,NA,0.2
134244,264413,1,2133-03-27 16:31:37,2133-04-04 16:15:28,2133-03-31,NA,2.748114,0,27.37778,⋯,4,2,3,1507.392,-1400.000,NA,2907.392,NA,NA,-0.1


In [6]:
nrow(subgroup)

[1] 6361

In [7]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [8]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [9]:
subgroup <- subgroup %>%
left_join(full_data %>% select(icustay_id, ps_weight, ps), by = "icustay_id") %>%
mutate(dobutamine_flag = as.factor(dobutamine_flag))
head(subgroup)

hadm_id,icustay_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,dobutamine_flag,vasofreeday28,⋯,sofa_drop_3,day1,day2,day3,down2,down3,lactate_diff,creatinine_diff,ps_weight,ps
159895,228416,1,2151-06-29 01:18:27,2151-07-27 16:00:56,2151-06-30,2151-09-18 00:00:00,0.000000,0,28.00000,⋯,0,-2110.000,-1930.000,-1475.000,-180.000,-635.000,-0.2,-0.7,1.809879,0.5525232
169700,278148,0,2197-01-09 00:45:42,2197-01-10 06:46:29,NA,2197-01-10 03:15:00,0.000000,0,0.00000,⋯,8,NA,NA,NA,NA,NA,NA,NA,1.389779,0.2804613
103722,274174,1,2181-09-08 23:43:13,2181-09-26 16:52:02,2181-09-11,2182-11-10 00:00:00,0.000000,0,28.00000,⋯,2,-300.000,-1827.000,-10.000,1527.000,-290.000,NA,0.5,3.320031,0.3012020
170515,214954,1,2176-03-11 04:58:23,2176-03-15 18:00:48,2176-03-11,NA,7.977352,0,26.04167,⋯,5,4299.590,2899.645,-1283.263,1399.945,5582.853,NA,0.2,1.252688,0.7982833
183493,222457,0,2105-06-04 20:57:55,2105-06-06 20:01:51,NA,NA,0.000000,0,28.00000,⋯,1,5988.072,2102.761,NA,3885.311,NA,NA,0.2,1.289399,0.2244447
134244,264413,1,2133-03-27 16:31:37,2133-04-04 16:15:28,2133-03-31,NA,2.748114,0,27.37778,⋯,3,1507.392,-1400.000,NA,2907.392,NA,NA,-0.1,3.011602,0.3320492


In [10]:
features <- c("ventfreeday28", "vasofreeday28", "dobutamine_flag", "day1", "day2", "day3",
              "sofa_drop_2", "sofa_drop_3",
              "norepinephrine_max", "lactate_diff", "creatinine_diff")
features

[1] "ventfreeday28"      "vasofreeday28"      "dobutamine_flag"   
 [4] "day1"               "day2"               "day3"              
 [7] "sofa_drop_2"        "sofa_drop_3"        "norepinephrine_max"
[10] "lactate_diff"       "creatinine_diff"

In [11]:
missing <- subgroup %>%
select(features) %>%
summarize_all(funs(sum(is.na(.)) / n())) %>%
gather("cov", "na") %>%
mutate(na = (na * 100) %>% round(1) %>% paste0("%"))

missing

cov,na
ventfreeday28,0%
vasofreeday28,0%
dobutamine_flag,0%
day1,10.7%
day2,18.9%
day3,34.4%
sofa_drop_2,0%
sofa_drop_3,0%
norepinephrine_max,0%
lactate_diff,71.6%


In [12]:
tab <- CreateTableOne(vars = features, strata = "echo",
                      data = subgroup,
                      argsNormal = list(var.equal = FALSE))
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [13]:
tab_df

,0,1,p,SMD
n,3099,3262,,
ventfreeday28 (mean (sd)),19.09 (13.53),18.02 (25.13),0.033,0.053
vasofreeday28 (mean (sd)),20.25 (12.61),20.13 (14.91),0.741,0.008
dobutamine_flag = 1 (%),23 (0.7),137 (4.2),<0.001,0.224
day1 (mean (sd)),1939.14 (3181.88),2456.26 (3853.10),<0.001,0.146
day2 (mean (sd)),835.26 (2429.91),1258.22 (2933.92),<0.001,0.157
day3 (mean (sd)),256.15 (2108.02),686.56 (2614.26),<0.001,0.181
sofa_drop_2 (mean (sd)),1.22 (4.34),0.89 (3.39),0.001,0.086
sofa_drop_3 (mean (sd)),2.29 (4.49),1.49 (4.07),<0.001,0.186
norepinephrine_max (mean (sd)),0.81 (2.39),1.78 (5.69),<0.001,0.221


In [14]:
ipw_svydesign <- survey::svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = subgroup)

In [15]:
tab_wtd <- svyCreateTableOne(vars = features,
                             strata = "echo",
                             data = ipw_svydesign)
capture.output(tab_wtd_df <- tab_wtd %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [16]:
tab_wtd_df

,0,1,p,SMD
n,5739.48,5842.36,,
ventfreeday28 (mean (sd)),14.96 (16.33),16.18 (36.90),0.903,0.003
vasofreeday28 (mean (sd)),18.35 (13.77),19.86 (20.23),0.047,0.054
dobutamine_flag = 1 (%),62.0 (1.1),196.3 (3.4),<0.001,0.155
day1 (mean (sd)),3523.72 (3747.62),3889.22 (4421.38),0.001,0.096
day2 (mean (sd)),2256.41 (2739.25),2484.92 (3305.25),<0.001,0.132
day3 (mean (sd)),1018.42 (2558.93),1683.87 (3066.01),<0.001,0.179
sofa_drop_2 (mean (sd)),1.00 (2.84),0.55 (3.03),<0.001,0.128
sofa_drop_3 (mean (sd)),1.36 (3.05),0.78 (3.57),<0.001,0.247
norepinephrine_max (mean (sd)),1.76 (3.26),3.05 (8.49),<0.001,0.154


In [17]:
ps_matches_df <- data.table::fread(file.path(data_dir, "ps_matches_df.csv"), data.table = FALSE)
head(ps_matches_df)

icustay_id,group,match
262436,ctrl,1
244460,ctrl,2
298177,ctrl,3
290165,ctrl,4
278397,ctrl,5
212022,ctrl,6


In [18]:
ps_df <- ps_matches_df %>% left_join(subgroup, by = "icustay_id")
head(ps_df)

icustay_id,group,match,hadm_id,echo,intime,outtime,echo_time,deathtime,norepinephrine_max,⋯,sofa_drop_3,day1,day2,day3,down2,down3,lactate_diff,creatinine_diff,ps_weight,ps
262436,ctrl,1,133275,0,2173-12-07 15:36:30,2173-12-09 21:27:39,NA,2173-12-09 18:30:00,4.46880,⋯,-15,NA,NA,NA,NA,NA,NA,NA,2.235326,0.5526379
244460,ctrl,2,177445,0,2143-12-27 20:59:00,2143-12-29 20:04:00,NA,NA,3.79920,⋯,10,3122.2500,1665.0000,NA,1457.250,NA,NA,0.3,1.430385,0.3008876
298177,ctrl,3,133566,0,2166-07-17 16:15:16,2166-07-22 18:41:28,NA,2166-07-27 00:00:00,1.50273,⋯,6,7910.2238,996.6645,-52.54445,6913.559,7962.768,NA,0.1,4.957195,0.7982730
290165,ctrl,4,197570,0,2129-12-24 05:33:11,2129-12-29 21:00:19,NA,NA,0.00000,⋯,2,6778.9266,3867.3365,2009.85431,2911.590,4769.072,NA,0.2,1.498298,0.3325758
278397,ctrl,5,194360,0,2187-08-28 19:40:43,2187-08-29 19:18:14,NA,NA,0.00000,⋯,2,-457.2597,NA,NA,NA,NA,NA,0.4,4.070197,0.7543116
212022,ctrl,6,159164,0,2141-03-23 19:59:04,2141-03-26 10:52:57,NA,2141-04-12 00:02:00,0.00000,⋯,0,740.4503,2080.1075,-359.61866,-1339.657,1100.069,NA,0.2,1.372052,0.2711645


In [19]:
tab_ps_pval_df <- features %>% map(function(feature) {
    df <- ps_df %>%
    select(!! as.name(feature), match, echo) %>%
    filter(!is.na(!! as.name(feature)))
    matched <- (df %>% filter(echo == 0) %>% rename(ctrl = (!! as.name(feature))) %>% select(ctrl, match)) %>%
    inner_join(df %>% filter(echo == 1) %>% rename(trtd = (!! as.name(feature))) %>% select(trtd, match),
              by = "match") %>%
    filter(!is.na(ctrl), !is.na(trtd))
    
    if (is.factor(ps_df %>% pull(feature))) {
        tab <- table(matched$trtd, matched$ctrl, dnn = c("Echo", "Non-Echo"))
        tst <- mcnemar.test(tab)
    } else {
        tst <- t.test(matched$trtd, matched$ctrl, paired = TRUE)
    }
    
    data.frame(cov = feature, pval = tst$p.val, stringsAsFactors = FALSE)
}) %>%
data.table::rbindlist() %>%
as.data.frame(stringsAsFactors = FALSE) %>%
mutate(pval = case_when(
    pval < 0.001 ~ "<0.001",
    TRUE ~ as.character(round(pval, 3))
)) %>%
rbind(data.frame(cov = "n", pval = "", stringsAsFactors = FALSE), .) %>%
setNames(c("cov", "p value (PS matching)"))

tab_ps_pval_df

cov,p value (PS matching)
n,
ventfreeday28,0.089
vasofreeday28,0.004
dobutamine_flag,0.077
day1,0.047
day2,0.002
day3,<0.001
sofa_drop_2,<0.001
sofa_drop_3,<0.001
norepinephrine_max,<0.001


In [20]:
tab_ps <- CreateTableOne(vars = features, strata = "echo", data = ps_df)
capture.output(tab_ps_df <- tab_ps %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [21]:
tab_ps_df

,0,1,p,SMD
n,1626,1626,,
ventfreeday28 (mean (sd)),18.10 (14.70),19.60 (32.64),0.090,0.059
vasofreeday28 (mean (sd)),19.31 (12.73),20.81 (16.95),0.004,0.100
dobutamine_flag = 1 (%),20 (1.2),34 (2.1),0.074,0.067
day1 (mean (sd)),2112.35 (3372.21),2432.39 (3768.86),0.016,0.089
day2 (mean (sd)),900.41 (2557.54),1275.30 (2872.91),<0.001,0.138
day3 (mean (sd)),253.25 (2147.55),771.78 (2683.65),<0.001,0.213
sofa_drop_2 (mean (sd)),1.31 (4.55),0.68 (3.26),<0.001,0.159
sofa_drop_3 (mean (sd)),2.46 (4.76),1.15 (3.99),<0.001,0.299
norepinephrine_max (mean (sd)),1.04 (2.68),1.38 (3.13),0.001,0.117


In [22]:
tab_fmt <- tab_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval, smd) %>%
setNames(c("cov", "Non-Echo", "Echo", "p value", "SMD"))

tab_fmt

cov,Non-Echo,Echo,p value,SMD
n,3099,3262,,
ventfreeday28 (mean (sd)),19.09 (13.53),18.02 (25.13),0.033,0.053
vasofreeday28 (mean (sd)),20.25 (12.61),20.13 (14.91),0.741,0.008
dobutamine_flag = 1 (%),0.7%,4.2%,<0.001,0.224
day1 (mean (sd)),1939.14 (3181.88),2456.26 (3853.10),<0.001,0.146
day2 (mean (sd)),835.26 (2429.91),1258.22 (2933.92),<0.001,0.157
day3 (mean (sd)),256.15 (2108.02),686.56 (2614.26),<0.001,0.181
sofa_drop_2 (mean (sd)),1.22 (4.34),0.89 (3.39),0.001,0.086
sofa_drop_3 (mean (sd)),2.29 (4.49),1.49 (4.07),<0.001,0.186
norepinephrine_max (mean (sd)),0.81 (2.39),1.78 (5.69),<0.001,0.221


In [23]:
tab_wtd_fmt <- tab_wtd_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval, smd) %>%
setNames(c("cov", "Non-Echo (weighted cohort)", "Echo (weighted cohort)",
           "p value (IPW)", "SMD (weighted cohort)"))

tab_wtd_fmt

cov,Non-Echo (weighted cohort),Echo (weighted cohort),p value (IPW),SMD (weighted cohort)
n,5739.48,5842.36,,
ventfreeday28 (mean (sd)),14.96 (16.33),16.18 (36.90),0.903,0.003
vasofreeday28 (mean (sd)),18.35 (13.77),19.86 (20.23),0.047,0.054
dobutamine_flag = 1 (%),1.1%,3.4%,<0.001,0.155
day1 (mean (sd)),3523.72 (3747.62),3889.22 (4421.38),0.001,0.096
day2 (mean (sd)),2256.41 (2739.25),2484.92 (3305.25),<0.001,0.132
day3 (mean (sd)),1018.42 (2558.93),1683.87 (3066.01),<0.001,0.179
sofa_drop_2 (mean (sd)),1.00 (2.84),0.55 (3.03),<0.001,0.128
sofa_drop_3 (mean (sd)),1.36 (3.05),0.78 (3.57),<0.001,0.247
norepinephrine_max (mean (sd)),1.76 (3.26),3.05 (8.49),<0.001,0.154


In [24]:
tab_ps_fmt <- tab_ps_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, smd) %>%
setNames(c("cov", "Non-Echo (PS matching)", "Echo (PS matching)", "SMD (PS matching)"))

tab_ps_fmt

cov,Non-Echo (PS matching),Echo (PS matching),SMD (PS matching)
n,1626,1626,
ventfreeday28 (mean (sd)),18.10 (14.70),19.60 (32.64),0.059
vasofreeday28 (mean (sd)),19.31 (12.73),20.81 (16.95),0.100
dobutamine_flag = 1 (%),1.2%,2.1%,0.067
day1 (mean (sd)),2112.35 (3372.21),2432.39 (3768.86),0.089
day2 (mean (sd)),900.41 (2557.54),1275.30 (2872.91),0.138
day3 (mean (sd)),253.25 (2147.55),771.78 (2683.65),0.213
sofa_drop_2 (mean (sd)),1.31 (4.55),0.68 (3.26),0.159
sofa_drop_3 (mean (sd)),2.46 (4.76),1.15 (3.99),0.299
norepinephrine_max (mean (sd)),1.04 (2.68),1.38 (3.13),0.117


In [25]:
tab_full <-
tab_fmt %>%
full_join(tab_wtd_fmt, by = "cov") %>%
full_join(tab_ps_fmt, by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
full_join(tab_ps_pval_df, by = "cov") %>%
full_join(missing %>% setNames(c("cov", "Missing data")), by = "cov") %>%
mutate(cov = case_when(
    cov == "ventfreeday28" ~ "Ventilation free days in 28 days",
    cov == "vasofreeday28" ~ "Vasopressor free days in 28 days",
    cov == "dobutamine_flag" ~ "Dobutamine Use",
    cov == "day1" ~ "IV Fluid Day 1 (mL)",
    cov == "day2" ~ "IV Fluid Day 2 (mL)",
    cov == "day3" ~ "IV Fluid Day 3 (mL)",
    cov == "sofa_drop_2" ~ "SOFA Reduction Day 2",
    cov == "sofa_drop_3" ~ "SOFA Reduction Day 3",
    cov == "norepinephrine_max" ~ "Norepinephrine (maximum dosage mg/min)",
    cov == "lactate_diff" ~ "Serum Lactate Reduction",
    cov == "creatinine_diff" ~ "Serum Creatinine Reduction",
    TRUE ~ cov
))

tab_full

cov,Non-Echo,Echo,p value,SMD,Non-Echo (weighted cohort),Echo (weighted cohort),p value (IPW),SMD (weighted cohort),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching),Missing data
n,3099,3262,,,5739.48,5842.36,,,1626,1626,,,NA
Ventilation free days in 28 days,19.09 (13.53),18.02 (25.13),0.033,0.053,14.96 (16.33),16.18 (36.90),0.903,0.003,18.10 (14.70),19.60 (32.64),0.059,0.089,0%
Vasopressor free days in 28 days,20.25 (12.61),20.13 (14.91),0.741,0.008,18.35 (13.77),19.86 (20.23),0.047,0.054,19.31 (12.73),20.81 (16.95),0.100,0.004,0%
Dobutamine Use,0.7%,4.2%,<0.001,0.224,1.1%,3.4%,<0.001,0.155,1.2%,2.1%,0.067,0.077,0%
IV Fluid Day 1 (mL),1939.14 (3181.88),2456.26 (3853.10),<0.001,0.146,3523.72 (3747.62),3889.22 (4421.38),0.001,0.096,2112.35 (3372.21),2432.39 (3768.86),0.089,0.047,10.7%
IV Fluid Day 2 (mL),835.26 (2429.91),1258.22 (2933.92),<0.001,0.157,2256.41 (2739.25),2484.92 (3305.25),<0.001,0.132,900.41 (2557.54),1275.30 (2872.91),0.138,0.002,18.9%
IV Fluid Day 3 (mL),256.15 (2108.02),686.56 (2614.26),<0.001,0.181,1018.42 (2558.93),1683.87 (3066.01),<0.001,0.179,253.25 (2147.55),771.78 (2683.65),0.213,<0.001,34.4%
SOFA Reduction Day 2,1.22 (4.34),0.89 (3.39),0.001,0.086,1.00 (2.84),0.55 (3.03),<0.001,0.128,1.31 (4.55),0.68 (3.26),0.159,<0.001,0%
SOFA Reduction Day 3,2.29 (4.49),1.49 (4.07),<0.001,0.186,1.36 (3.05),0.78 (3.57),<0.001,0.247,2.46 (4.76),1.15 (3.99),0.299,<0.001,0%
Norepinephrine (maximum dosage mg/min),0.81 (2.39),1.78 (5.69),<0.001,0.221,1.76 (3.26),3.05 (8.49),<0.001,0.154,1.04 (2.68),1.38 (3.13),0.117,<0.001,0%


In [26]:
tab_report <- tab_full %>% select(1, 10, 11, 12, 13)
tab_report

cov,Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching)
n,1626,1626,,
Ventilation free days in 28 days,18.10 (14.70),19.60 (32.64),0.059,0.089
Vasopressor free days in 28 days,19.31 (12.73),20.81 (16.95),0.100,0.004
Dobutamine Use,1.2%,2.1%,0.067,0.077
IV Fluid Day 1 (mL),2112.35 (3372.21),2432.39 (3768.86),0.089,0.047
IV Fluid Day 2 (mL),900.41 (2557.54),1275.30 (2872.91),0.138,0.002
IV Fluid Day 3 (mL),253.25 (2147.55),771.78 (2683.65),0.213,<0.001
SOFA Reduction Day 2,1.31 (4.55),0.68 (3.26),0.159,<0.001
SOFA Reduction Day 3,2.46 (4.76),1.15 (3.99),0.299,<0.001
Norepinephrine (maximum dosage mg/min),1.04 (2.68),1.38 (3.13),0.117,<0.001


In [27]:
data.table::fwrite(tab_full, file.path(data_dir, "subgroup.csv"))
data.table::fwrite(tab_report, file.path(data_dir, "subgroup_report.csv"))